In [1]:
#import packages and functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import fiona

import shapely
from shapely.validation import make_valid


In [2]:
from general_functions import *

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#define relative paths
waterschap = 'Scheldestromen'
path_Scheldestromen = '..\..\Data_preprocessed\Waterschappen\Scheldestromen\Scheldestromen.gpkg'
output_gpkg_path = "../../Data_postprocessed/Waterschappen/Scheldestromen"


In [5]:
Scheldestromen = read_gpkg_layers(gpkg_path = path_Scheldestromen, 
                                  variables = ['stuw', 
                                               'gemaal', 
                                               # 'afsluitmiddel',
                                               'duikersifonhevel',
                                               'hydroobject',
                                               # 'peilgebiedvigerend',
                                               # 'peilgebiedpraktijk',
                                               # 'peilafwijkinggebied',
                                               # 'streefpeil',
                                              ],
                                 engine = 'pyogrio') 
                               
#the peilgebieden and streefpeilen do not contain overlapping values. Scheldestromen has delivered additional files as shapes
Scheldestromen['peilgebied'] = gpd.read_file('..\..\Data_preprocessed\Waterschappen\Scheldestromen\Praktijkpeilgebieden_20231204.shp')

In [6]:
Scheldestromen['peilgebied']['code'] = Scheldestromen['peilgebied']['GPGIDENT']
Scheldestromen['peilgebied']['globalid'] = Scheldestromen['peilgebied']['GLOBALID'] 
Scheldestromen['peilgebied']['waterhoogte'] = Scheldestromen['peilgebied']['GPGZP'] 
Scheldestromen['peilgebied']['nen3610id'] = 'dummy_nen3610id_peilgebied_' + Scheldestromen['peilgebied'].index.astype(str)

Scheldestromen['streefpeil'] = gpd.GeoDataFrame()
Scheldestromen['streefpeil']['waterhoogte'] = Scheldestromen['peilgebied']['waterhoogte']
Scheldestromen['streefpeil']['globalid'] = Scheldestromen['peilgebied']['globalid']
Scheldestromen['streefpeil']['geometry'] = np.nan


Scheldestromen['peilgebied'] = Scheldestromen['peilgebied'][['code', 'nen3610id', 'globalid', 'geometry']]

C:\Users\Bruijns\AppData\Local\Temp\ipykernel_2280\2889691104.py:9: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  Scheldestromen['streefpeil']['geometry'] = np.nan


In [7]:
#convert multiz points to points
Scheldestromen['stuw'].geometry = Scheldestromen['stuw'].centroid
Scheldestromen['gemaal'].geometry = Scheldestromen['gemaal'].centroid


# Scheldestromen

In [8]:
#stuw
Scheldestromen['stuw'] = Scheldestromen['stuw'][['code', 'nen3610id', 'globalid', 'geometry']]

#gemaal
Scheldestromen['gemaal'] = Scheldestromen['gemaal'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['gemaal']['code'] = 'dummy_code_gemaal_' + Scheldestromen['gemaal'].index.astype(str)

#hydroobject
Scheldestromen['hydroobject']['code'] = Scheldestromen['hydroobject']['naam']
Scheldestromen['hydroobject'] = Scheldestromen['hydroobject'][['code', 'nen3610id', 'globalid', 'geometry']]

#duikersifonhevel
Scheldestromen['duikersifonhevel'] = Scheldestromen['duikersifonhevel'][['code', 'nen3610id', 'globalid', 'geometry']]


In [9]:
# pd.merge(left = Scheldestromen['peilgebiedpraktijk'],
#          right = Scheldestromen['streefpeil'],
#          left_on = 'globalid',
#          right_on = 'peilgebiedpraktijkid')

In [10]:
Scheldestromen['stuw'] = Scheldestromen['stuw'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['gemaal'] = Scheldestromen['gemaal'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['hydroobject'] = Scheldestromen['hydroobject'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['duikersifonhevel'] = Scheldestromen['duikersifonhevel'][['code', 'nen3610id', 'globalid', 'geometry']]


In [11]:
# Scheldestromen['peilgebiedpraktijk']['geometry'] = Scheldestromen['peilgebiedpraktijk'].buffer(distance = 0)
# Scheldestromen['peilafwijkinggebied']['geometry'] = Scheldestromen['peilafwijkinggebied'].buffer(distance = 0)

In [12]:
# peilgebied = pd.merge(left = Scheldestromen['streefpeil'],
#                       right = Scheldestromen['peilgebiedpraktijk'],
#                       left_on = 'peilgebiedpraktijkid',
#                       right_on = 'globalid')

In [13]:
# streefpeil = gpd.GeoDataFrame()
# streefpeil['waterhoogte'] = peilgebied['waterhoogte']
# streefpeil['globalid'] = peilgebied['peilgebiedpraktijkid']
# streefpeil['geometry'] = np.nan
# Scheldestromen['streefpeil'] = gpd.GeoDataFrame(streefpeil)

In [14]:
# Scheldestromen['peilgebied'] = gpd.GeoDataFrame()
# Scheldestromen['peilgebied'][['code', 'nen3610id', 'globalid', 'geometry']] = peilgebied[['code', 'nen3610id_y', 'globalid_y', 'geometry_y']]

In [15]:
#delete irrelvant data
variables = ['peilgebiedpraktijk', 'peilgebiedvigerend', 'peilafwijkinggebied']

for variable in variables:
    if str(variable) in Scheldestromen:
        del Scheldestromen[variable]

### Check for the correct keys and columns

In [16]:
show_layers_and_columns(waterschap = Scheldestromen)

stuw
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

gemaal
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

duikersifonhevel
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

hydroobject
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

peilgebied
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

streefpeil
['waterhoogte' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  None



In [18]:
for i in range(len(Scheldestromen['peilgebied'])):    
    if type(Scheldestromen['peilgebied'].loc[i, 'geometry']) == Polygon:
        Scheldestromen['peilgebied'].loc[i, 'geometry'].plot()

<class 'shapely.geometry.polygon.Polygon'>


NameError: name 'Polygon' is not defined

In [38]:
merged = pd.merge(left = Scheldestromen['peilgebied'],
         right = Scheldestromen['streefpeil'],
         on = 'globalid')

merged[merged.waterhoogte.isna()]


,code,nen3610id,globalid,geometry_x,waterhoogte,geometry_y


### Store data

In [82]:
# Check if the directory exists
if not os.path.exists(output_gpkg_path):
    # If it doesn't exist, create it
    os.makedirs(output_gpkg_path)
    
store_data(waterschap = Scheldestromen, 
           output_gpkg_path = output_gpkg_path + '/Scheldestromen')


In [92]:
Scheldestromen['hydroobject']

,code,nen3610id,globalid,geometry
0,OAF42854,NL.WBHCODE.42.Hydroobject.1,{cc6dc9d0-2a73-4dc6-8cca-38479d5415d5},MULTILINESTRING Z ((43578.569 413520.087 0.000...
1,OAF30774,NL.WBHCODE.42.Hydroobject.2,{087f93c5-3208-46e2-93c7-31e33ee8b10f},MULTILINESTRING Z ((24655.211 392203.392 0.000...
2,OAF31525,NL.WBHCODE.42.Hydroobject.3,{ef5ccc42-aca5-496d-9854-aa56319ee87e},MULTILINESTRING Z ((30657.892 399818.261 0.000...
3,OAF47117,NL.WBHCODE.42.Hydroobject.4,{332a351e-d193-4022-acfb-d781dee87fbd},MULTILINESTRING Z ((31724.490 385131.330 0.000...
4,OAF75952,NL.WBHCODE.42.Hydroobject.5,{e190b7ba-7cd8-42ad-805a-84f69714d3b9},MULTILINESTRING Z ((52163.586 359474.482 0.000...
...,...,...,...,...
70928,OAF104394,NL.WBHCODE.42.Hydroobject.70929,{169a186b-4b3e-461c-84a2-770198b1f889},MULTILINESTRING Z ((30048.913 398032.745 0.000...
70929,OAF104397,NL.WBHCODE.42.Hydroobject.70930,{dab8f9a5-6eea-4996-8c5e-0a40b51c6e7f},MULTILINESTRING Z ((30055.697 398032.266 0.000...
70930,OAF104398,NL.WBHCODE.42.Hydroobject.70931,{09d88aa3-190d-4e39-bc27-8b473a1f5ddb},MULTILINESTRING Z ((30057.368 398056.163 0.000...
70931,OAF103808,NL.WBHCODE.42.Hydroobject.70932,{0327d41d-2a6a-4c4e-9a88-df1f01c3438a},MULTILINESTRING Z ((32377.674 397795.063 0.000...
